In [35]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Load the train data
train_data = pd.read_csv(r"C:\Users\DELL\Downloads\train_LZdllcl.csv")

# Handle missing values
train_data['previous_year_rating'] = train_data['previous_year_rating'].fillna(train_data['previous_year_rating'].mean())

# One-hot encode categorical columns
train_data = pd.get_dummies(train_data, columns=['department', 'region', 'education', 'gender', 'recruitment_channel'])

# Select features and target
train_features = train_data.drop('avg_training_score', axis=1)
train_target = train_data['avg_training_score']

# Scale the features
scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features)



In [37]:
# Load the test data
test_data = pd.read_csv(r"C:\Users\DELL\Downloads\test_2umaH9m.csv")

# Handle missing values in the test data
test_data['previous_year_rating'] = test_data['previous_year_rating'].fillna(test_data['previous_year_rating'].mean())

# One-hot encode categorical columns in the test data
test_data = pd.get_dummies(test_data, columns=['department', 'region', 'education', 'gender', 'recruitment_channel'])

# Ensure the test data has the same columns as the training data
missing_cols = set(train_features.columns) - set(test_data.columns)
for col in missing_cols:
    test_data[col] = 0
test_data = test_data[train_features.columns]

# Scale the test data
test_features_scaled = scaler.transform(test_data)



In [39]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train_features_scaled, train_target, test_size=0.2, random_state=42)



In [41]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV

# Define the corrected parameter grid
param_grid = {
    'fit_intercept': [True, False],
    'copy_X': [True, False],
    'n_jobs': [None, -1]
}

# Initialize the GridSearchCV object
grid_search = GridSearchCV(LinearRegression(), param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the model
try:
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_
    print("Best parameters:", best_params)

    # Train the model with the best parameters
    best_model = LinearRegression(**best_params)
    best_model.fit(X_train, y_train)
except ValueError as e:
    print("Error during model fitting:", e)
    best_model = LinearRegression()
    best_model.fit(X_train, y_train)



Best parameters: {'copy_X': True, 'fit_intercept': True, 'n_jobs': None}


In [43]:
from sklearn.metrics import mean_squared_error

y_pred = best_model.predict(X_val)
print("Validation MSE after tuning:", mean_squared_error(y_val, y_pred))


Validation MSE after tuning: 16.62078173686298


In [45]:
test_predictions = best_model.predict(test_features_scaled)
test_predictions = (test_predictions > 0.5).astype(int)


In [47]:
# Load the sample submission file
submission = pd.read_csv(r"C:\Users\DELL\Downloads\sample_submission_M0L0uXE.csv")

# Update the submission file with predictions
submission['is_promoted'] = test_predictions

# Save the submission file
submission.to_csv('final_submission.csv', index=False)


In [53]:
import os
print(os.getcwd())

C:\Users\DELL


In [55]:
imported_df = pd.read_csv('final_submission.csv')
print(imported_df)

       employee_id  is_promoted
0             8724            1
1            74430            1
2            72255            1
3            38562            1
4            64486            1
...            ...          ...
23485        53478            1
23486        25600            1
23487        45409            1
23488         1186            1
23489         5973            1

[23490 rows x 2 columns]


In [57]:
print(os.listdir(os.getcwd()))

['.anaconda', '.conda', '.condarc', '.continuum', '.ipynb_checkpoints', '.ipython', '.jupyter', '.matplotlib', '.ms-ad', '1.ipynb', '3D Objects', 'anaconda3', 'AppData', 'Application Data', 'Assignment on Classification.ipynb', 'Assignment on Clustering.ipynb', 'Bank Account Manager.ipynb', 'Conditional Statements.ipynb', 'Contacts', 'Cookies', 'Correlation.ipynb', 'DATA STRUCTURES.ipynb', 'Documents', 'Downloads', 'Encoding.ipynb', 'Favorites', 'Feature Engineering.ipynb', 'final_submission.csv', 'Handling missing values.ipynb', 'IntelGraphicsProfiles', 'Intermediate assessment 1.ipynb', 'Intermediate assessment 2.ipynb', 'Linear Regression.ipynb', 'Links', 'Local Settings', 'Matplotlib and Seaborn.ipynb', 'Microsoft', 'MicrosoftEdgeBackups', 'Music', 'My Documents', 'NetHood', 'NTUSER.DAT', 'ntuser.dat.LOG1', 'ntuser.dat.LOG2', 'NTUSER.DAT{15a0d507-20cb-11ef-b887-e02c70fcce20}.TM.blf', 'NTUSER.DAT{15a0d507-20cb-11ef-b887-e02c70fcce20}.TMContainer00000000000000000001.regtrans-ms', 'NT